# setting lib

In [ ]:
import torch

print("PyTorch 버전:", torch.__version__)
print("GPU 사용 가능 여부:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU 이름:", torch.cuda.get_device_name(0))
    
 # CUDA 12.1
# conda install pytorch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 pytorch-cuda=12.1 -c pytorch -c nvidia


PyTorch 버전: 2.1.0+cu118
GPU 사용 가능 여부: True
GPU 이름: NVIDIA RTX A6000


In [ ]:
# !pip install ipywidgets
# !python --upgrade pip #25.0.1
# !pip install --upgrade jupyter jupyterlab
# !pip install torch transformers accelerate
# !pip install --upgrade transformers
# !pip install onnxruntime onnxruntime-tools
# !pip install transformers torch onnx onnxruntime



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.1
    Uninstalling pip-23.3.1:
      Successfully uninstalled pip-23.3.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 72.2 MB/s eta 0:00:00
  Attempting uninstall: jupyterlab-server
    Found existing installation: jupyterlab_server 2.25.0
    Uninstalling jupyterlab_server-2.25.0:
      Successfully uninstalled jupyterlab_server-2.25.0
  Attempting uninstall: jupyterlab
    Found existing installation: jupyterlab 4.0.8
    Uninstalling jupyterlab-4.0.8:
      Successfully uninstalled jupyterlab-4.0.8
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import sys
import torch
import transformers
import onnx
import onnxruntime
import tokenizers
import numpy

import re
import time
import json


print("🧩 Python version:", sys.version.split()[0])
print("🧩 PyTorch version:", torch.__version__)
print("🧩 Transformers version:", transformers.__version__)
print("🧩 ONNX version:", onnx.__version__)
print("🧩 ONNX Runtime version:", onnxruntime.__version__)
print("🧩 Tokenizers version:", tokenizers.__version__)
print("🧩 NumPy version:", numpy.__version__)
print("🧩 NumPy version:", optimum.__version__)


🧩 Python version: 3.10.12
🧩 PyTorch version: 2.1.0+cu118
🧩 Transformers version: 4.51.3
🧩 ONNX version: 1.17.0
🧩 ONNX Runtime version: 1.21.1
🧩 Tokenizers version: 0.21.1
🧩 NumPy version: 1.24.1


In [ ]:
import torch
torch.cuda.empty_cache()

# phi

In [18]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

phi_tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct", trust_remote_code=True)
phi= AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-4k-instruct", trust_remote_code=True,torch_dtype=torch.float16).to("cuda")


tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


ModuleNotFoundError: No module named 'torchvision'

## phi 프롬프트

In [ ]:
# prompt_templates.py

## Zero-shot

def prompt_a(input_text):
    return f"""
다음 문장에서 사용자의 의도(intent), 필요한 정보(slot), 그리고 자연스러운 응답 메시지(response)를 추출하세요.  
출력은 아래 형식을 따르세요:

- intent는 다음 중 하나입니다: transfer, inquiry, confirm, cancel, other,system_response

{{
  "intent": "...",
  "slots": {{ "recipient": "...", "amount": ... }},
  "response": "..."
}}

문장: "{input_text}"
"""


def prompt_b(input_text):
    return f"""
    다음 문장에서 사용자의 의도(intent), 필요한 정보(slot), 그리고 응답 메시지를 추출하세요.
    의도(intent)는 아래 분류 중 하나로만 선택해야 합니다:
    다음 문장을 분석하여 intent, amount, recipient, response를 JSON형식으로 출력해 주세요.

    - transfer: 금전을 누군가에게 보내려는 요청
    - inquiry: 잔액, 수수료 등 정보 문의
    - cancel: 진행 중인 작업 취소
    - confirm: 이전 발화에 대한 확인
    - other: 시스템 외 일상 대화 또는 분류 불가능
    - system_response: 시스템이 사용자에게 안내하거나 재질문

    출력은 아래 JSON 형식을 따르세요:

{{
  "intent": "...",
  "slots": {{"recipient": "...", "amount": ... }},
  "response": "..."
}}

문장: "{input_text}"
"""
#one-shot
def prompt_c(input_text):
    return f"""
    다음 문장을 분석하여 intent, amount, recipient, response를 예시 형식을 따라 추출해 주세요.
    intent는 다음 중 하나입니다: transfer, inquiry, confirm, cancel, other,system_response
    response는 고객님에게 안내할 자연스러운 한국어 문장을 제시하세요

예시:
text: "엄마한테 삼만원 보내줘"

  {{"intent": "transfer","amount": 30000,"recipient": "엄마","response": "엄마님께 30,000원을 송금해드릴까요?"}}


{input_text}
"""

def prompt_d(input_text):
    return f"""
    다음 문장을 분석하여 intent, amount, recipient, response를 예시 형식을 따라 추출해 주세요.

    intent는 다음 중 하나입니다: 
        transfer:사용자가 누군가에게 금전을 보내고자 하는 의도, 
        confirm:이전 발화를 확인하거나 반복하는 표현,
        cancel:이전 동작을 취소하거나 거절하려는 의도, 
        inquiry:송금과 관련된 정보를 묻거나 확인하는 의도, 
        other:시스템과 무관한 일상 대화 or 분류 불가능한 문장,
        system_response: 시스템이 사용자에게 재질문하거나 안내하는 응답 발화
    amount는 숫자만 (없으면 None)
    recipient는 사람 이름 등 (없으면 None)
    response는 고객님에게 안내할 자연스러운 한국어 문장을 제시하세요

예시:
text: "엄마한테 삼만원 보내줘"

  {{"intent": "transfer","amount": 30000,"recipient": "엄마","response": "엄마님께 30,000원을 송금해드릴까요?"}}
{input_text}
"""

def prompt_e(input_text):
    return f"""당신은 사용자의 금융 발화를 분석하는 AI 송금 도우미입니다. 다음 지침에 따라 작동하세요:

1. 사용자의 문장에서 다음 항목을 추출하세요:
    - intent: 다음 중 하나 (transfer, confirm, cancel, inquiry, other, system_response)
    - amount: 숫자만 추출 (없으면 null)
    - recipient: 사람 이름 등 (없으면 null)

2. 사용자의 발화에 어울리는 자연스러운 안내 응답(response)을 생성하세요.

3. 다음 JSON 형식으로 하나의 객체로 응답하세요. 다른 텍스트는 출력하지 마세요.

예시:

"엄마한테 삼만원 보내줘"

{{
  "intent": "transfer",
  "amount": 30000,
  "recipient": "엄마",
  "response": "엄마님께 30,000원을 송금해드릴까요?"
}}

{input_text}
"""

## 함수 정의

In [ ]:
##original

def run_inference(prompt: str, tokenizer, model):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    start = time.time()
    outputs = model.generate(**inputs, max_new_tokens=80,do_sample=False, use_cache=False)
    end = time.time()

    # 출력 해석
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    output_text = generated.replace(prompt, "").strip()


    try:
        json_start = output_text.find("{")
        json_end = output_text.rfind("}") + 1
        json_block = output_text[json_start:json_end]
        parsed = json.loads(json_block)
        
    except Exception as e:
        print("JSON 파싱 실패:", e)
        print("📦 원문 출력:", repr(output_text))
        parsed = {}

    print("\n📦 LLM 원문 출력:")
    print(output_text)
    print(f"\n⏱️ 처리 시간: {round(end - start, 2)}초")
    return parsed, round(end - start, 2)

In [ ]:
## new_ver

def run_inference(prompt: str, tokenizer, model):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    start = time.time()
    outputs = model.generate(**inputs, max_new_tokens=80,do_sample=False, use_cache=False)
    end = time.time()

    # 출력 해석
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    output_text = generated.replace(prompt, "").strip()


    try:
        json_start = output_text.find("{")
        json_end = output_text.rfind("}") + 1
        parsed = json.loads(output_text[json_start:json_end])
        
    except Exception as e:
        print("JSON 파싱 실패:", e)
        print("원문 출력:", repr(output_text))
        parsed = {'fail'}

    print("\nLLM 원문 출력:")
    print(output_text)
    print(f"\n처리 시간: {round(end - start, 2)}초")
    return parsed, round(end - start, 2)

## 함수 test

In [ ]:
text = "아빠한테 오천 원 보내줘"
prompt = prompt_c(text)

# phi 인퍼런스 실행
phi_return,time = run_inference(prompt,phi_tokenizer, phi)
print("phi",phi_return)



AttributeError: 'DynamicCache' object has no attribute 'get_max_length'

In [ ]:
#파싱 방법 변경

import re

def extract_fields_by_keywords(text: str):
    """
    JSON 파싱 실패 시, response 키워드 등으로부터 값 추출 (정규표현식 기반)
    """
    try:
        intent = re.search(r'"?intent"?\s*:\s*"?(\w+)"?', text)
        amount = re.search(r'"?amount"?\s*:\s*(\d+)', text)
        recipient = re.search(r'"?recipient"?\s*:\s*"?([\w가-힣]+)"?', text)
        response = re.search(r'"?response"?\s*:\s*"?([^"]+)"?', text)

        return {
            "intent": intent.group(1) if intent else None,
            "amount": int(amount.group(1)) if amount else None,
            "recipient": recipient.group(1) if recipient else None,
            "response": response.group(1) if response else None
        }
    except Exception as e:
        print("키워드 기반 파싱 실패:", e)
        return {'fail'}

def run_inference(prompt: str, tokenizer, model):
    import time
    import json

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    start = time.time()
    outputs = model.generate(**inputs, max_new_tokens=80, do_sample=False, use_cache=False)
    end = time.time()

    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    output_text = generated.replace(prompt, "").strip()

    try:
        json_start = output_text.find("{")
        json_end = output_text.rfind("}") + 1
        parsed = json.loads(output_text[json_start:json_end])
    except Exception as e:
        print("🔴 JSON 파싱 실패:", e)
        print("📦 원문 출력:", repr(output_text))
        parsed = extract_fields_by_keywords(output_text)

    print("\n📦 LLM 원문 출력:")
    print(output_text)
    print(f"\n⏱️ 처리 시간: {round(end - start, 2)}초")
    return parsed, round(end - start, 2)


## Model Test

In [ ]:
samples = [
  {
    "text": "철수한테 오천 원 보내줘",
    "intent": "transfer",
    "slots": { "recipient": "철수", "amount": 5000 },
    "response": "철수님께 5,000원을 송금하시겠어요?"
  },
  {
    "text": "엄마한테 3만원만",
    "intent": "transfer",
    "slots": { "recipient": "엄마", "amount": 30000 },
    "response": "엄마님께 30,000원을 송금하시겠어요?"
  },
  {
    "text": "만원만 보내줘",
    "intent": "transfer",
    "slots": { "recipient": None, "amount": 10000 },
    "response": "누구에게 10,000원을 송금할까요?"
  },
  {
    "text": "엄마한테 보내줘",
    "intent": "transfer",
    "slots": { "recipient": "엄마", "amount": None },
    "response": "엄마님께 얼마를 송금할까요?"
  },
  {
    "text": "나 송금하고 싶어",
    "intent": "transfer",
    "slots": { "recipient": None, "amount": None },
    "response": "송금하실 대상과 금액을 알려주세요."
  },
  {
    "text": "가스비 내야 하는데 어떻게 해야 해?",
    "intent": "inquiry",
    "slots": { "recipient": None, "amount": None, "topic": "가스비" },
    "response": "가스비 납부 방법을 알려드릴게요."
  },
  {
    "text": "내 계좌에 얼마 남았어?",
    "intent": "inquiry",
    "slots": { "recipient": None, "amount": None, "topic": "잔액조회" },
    "response": "계좌 잔액을 확인 중입니다."
  },
  {
    "text": "그 사람한테 또 보내줘",
    "intent": "transfer",
    "slots": { "recipient": "이전대화", "amount": "이전대화" },
    "response": "이전에 송금한 대상과 금액으로 다시 송금할까요?"
  },
  {
    "text": "아까랑 똑같이 보내줘",
    "intent": "transfer",
    "slots": { "recipient": "이전대화", "amount": "이전대화" },
    "response": "이전과 동일한 내용으로 송금 진행할까요?"
  },
  {
    "text": "송금할래",
    "intent": "transfer",
    "slots": { "recipient": None, "amount": None },
    "response": "송금하실 대상과 금액을 말씀해주세요."
  },
  {
    "text": "이체 수수료 얼마야?",
    "intent": "inquiry",
    "slots": { "recipient": None, "amount": None, "topic": "이체 수수료" },
    "response": "이체 수수료 정보를 알려드릴게요."
  },
  {
    "text": "오늘 아빠랑 점심 먹었어",
    "intent": "other",
    "slots": { "recipient": None, "amount": None },
    "response": "말씀 감사합니다!"
  },
  {
    "text": "아빠한테 이만원 보내줘",
    "intent": "transfer",
    "slots": { "recipient": "아빠", "amount": 20000 },
    "response": "아빠님께 20,000원을 송금하시겠어요?"
  },
  {
    "text": "여보한테 십만 원 이체해줘",
    "intent": "transfer",
    "slots": { "recipient": "여보", "amount": 100000 },
    "response": "여보님께 100,000원을 송금하시겠어요?"
  },
  {
    "text": "보내지 마",
    "intent": "cancel",
    "slots": { "recipient": None, "amount": None },
    "response": "요청하신 송금을 취소했습니다."
  },
  {
    "text": "그냥 넘어가줘",
    "intent": "cancel",
    "slots": { "recipient": None, "amount": None },
    "response": "진행 중이던 요청을 취소했어요."
  },
  {
    "text": "엄마 송금해줘",
    "intent": "transfer",
    "slots": { "recipient": "엄마", "amount": None },
    "response": "엄마님께 얼마를 송금할까요?"
  },
  {
    "text": "아, 삼만원 보내는 거였지",
    "intent": "confirm",
    "slots": { "recipient": None, "amount": 30000 },
    "response": "30,000원 송금 요청으로 확인했습니다."
  },
  {
    "text": "예금해줘",
    "intent": "other",
    "slots": { "recipient": None, "amount": None, "action": "예금" },
    "response": "예금을 진행할까요? 금액을 알려주세요."
  },
  {
    "text": "누구한테 보낼까요?",
    "intent": "system_response",
    "slots": { "recipient": None, "amount": None },
    "response": "누구에게 송금할까요?"
  }
]

In [ ]:
#gemma 인퍼런스
gemma_return = run_inference(prompt,gemma_processor, gemma)
print("\ngemma",gemma_return)

### 파싱 실세 pase=fail 딕셔너리

In [ ]:


results = []
total_time = 0

for sample in samples:
    output, elapsed = run_inference(prompt_e(sample["text"]), phi_tokenizer, phi)
    total_time += elapsed

    if isinstance(output, dict):
        results.append({
            "text": sample["text"],
            "intent": output.get("intent", None),
            "slots": output.get("slots", {
                "recipient": None,
                "amount": None
            }),
            "response": output.get("response", ""),
            "_meta": {
                "inference_time": elapsed
            }
        })
    else:
        results.append({
            "text": sample["text"],
            "intent": None,
            "slots": {
                "recipient": None,
                "amount": None
            },
            "response": "",
            "_meta": {
                "inference_time": elapsed,
                "error": "Parsing failed"
            }
        })



avg_time = total_time / len(samples)
print(f"⏱️ 평균 추론 시간: {avg_time:.2f}초")

# 결과 저장
with open("results_e_new.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print("결과 저장 완료")

🔴 JSON 파싱 실패: Expecting value: line 1 column 1 (char 0)
📦 원문 출력: '{\n  "intent": "transfer",\n  "amount": 100000,\n  "recipient": "철수",\n  "response": "철수님께 100,000원을 송금해드릴까요?"'

📦 LLM 원문 출력:
{
  "intent": "transfer",
  "amount": 100000,
  "recipient": "철수",
  "response": "철수님께 100,000원을 송금해드릴까요?"

⏱️ 처리 시간: 5.34초

📦 LLM 원문 출력:
{
  "intent": "transfer",
  "amount": 30000,
  "recipient": "엄마",
  "response": "엄마님께 3만원을 송금해드릴까요?"
}����

⏱️ 처리 시간: 5.65초

📦 LLM 원문 출력:
{
  "intent": "transfer",
  "amount": 10000,
  "recipient": null,
  "response": "10,000원을 송금하시겠습니까?"
}

삼만원 보내��

⏱️ 처리 시간: 5.82초

📦 LLM 원문 출력:
{
  "intent": "transfer",
  "amount": 20000,
  "recipient": "엄마",
  "response": "엄마님께 20,000원을 송금해드릴까요?"
}

⏱️ 처리 시간: 6.04초

📦 LLM 원문 출력:
{
  "intent": "transfer",
  "amount": null,
  "recipient": null,
  "response": "몇 원을 송금하고 싶은가요?"
}

저희 예산 최대 송

⏱️ 처리 시간: 6.28초
🔴 JSON 파싱 실패: Expecting value: line 1 column 1 (char 0)
📦 원문 출력: '{\n  "intent": "inquiry",\n  "amount": null,\n  "recipie

## 평가

In [ ]:
# Load result_c.json
with open("/workspace/results_e_new.json", "r", encoding="utf-8") as f:
    results = json.load(f)


# 평가 지표 초기화
correct_intent = 0
correct_recipient = 0
correct_amount = 0
parsing_success = 0

for result, ex in zip(results, samples):
    meta = result.get("_meta", {})
    
    # 파싱 성공 여부
    if "error" not in meta:
        parsing_success += 1

    # 예측값
    pred_intent = result.get("intent")
    pred_recipient = result.get("slots", {}).get("recipient")
    pred_amount = result.get("slots", {}).get("amount")

    # 정답값
    true_intent = ex["intent"]
    true_recipient = ex["slots"]["recipient"]
    true_amount = ex["slots"]["amount"]

    # 평가
    if pred_intent == true_intent:
        correct_intent += 1
    if pred_recipient == true_recipient:
        correct_recipient += 1
    if pred_amount == true_amount:
        correct_amount += 1

# 총 샘플 수
total = len(samples)

# 평가 결과 정리
evaluation = {
    "Intent 정확도": f"{correct_intent}/{total} ({correct_intent/total:.0%})",
    "Recipient 정확도": f"{correct_recipient}/{total} ({correct_recipient/total:.0%})",
    "Amount 정확도": f"{correct_amount}/{total} ({correct_amount/total:.0%})",
    "파싱 성공률": f"{parsing_success}/{total} ({parsing_success/total:.0%})"
}

print(evaluation)

{'Intent 정확도': '15/20 (75%)', 'Recipient 정확도': '12/20 (60%)', 'Amount 정확도': '12/20 (60%)', '파싱 성공률': '20/20 (100%)'}


# QWEN

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
# !pip install prompt_templates

ImportError: cannot import name 'unified_system_prompt' from 'prompt_templates' (/usr/local/lib/python3.10/dist-packages/prompt_templates/__init__.py)

In [ ]:
qwen_tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")
qwen = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct",torch_dtype=torch.float16).to("cuda")

NameError: name 'AutoTokenizer' is not defined

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM 
import torch 
model_id = "Qwen/Qwen2.5-0.5B" 
tokenizer = AutoTokenizer.from_pretrained(model_id) 
model = AutoModelForCausalLM.from_pretrained(model_id, torchscript=True) 
model.eval()  
dummy_input = tokenizer("안녕", return_tensors="pt").input_ids  


In [ ]:
#qwen 프롬프트

# prompt_templates.py

def unified_system_prompt(input_text: str) -> list:
    """
    Qwen, GPT 등 ChatML 구조 기반 LLM에 입력할 메시지 포맷 (system + user).
    분석 + 응답을 동시에 요청합니다.
    """
    system_message = {
        "role": "system",
        "content": """
당신은 사용자의 금융 발화를 분석하는 AI 송금 도우미입니다. 다음 지침에 따라 작동하세요:

1. 사용자의 문장에서 다음 항목을 추출하세요:
    - intent: 다음 중 하나 (transfer, confirm, cancel, inquiry, other, system_response)
    - amount: 숫자만 추출 (없으면 null)
    - recipient: 사람 이름 등 (없으면 null)

2. 사용자의 발화에 어울리는 자연스러운 안내 응답(response)을 생성하세요.

3. 다음 JSON 형식으로 하나의 객체로 응답하세요. 다른 텍스트는 출력하지 마세요.

예시:
{
  "intent": "transfer",
  "amount": 30000,
  "recipient": "엄마",
  "response": "엄마님께 30,000원을 송금해드릴까요?"
}
"""
    }

    user_message = {
        "role": "user",
        "content": input_text
    }

    return [system_message, user_message]


In [ ]:
def run_inference_qwen(text: str, tokenizer, model):
    from prompt_templates import unified_system_prompt

    messages = unified_system_prompt(text)
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    start = time.time()
    outputs = model.generate(**inputs, max_new_tokens=128, do_sample=False, use_cache=False)
    end = time.time()

    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    output_text = generated.replace(prompt, "").strip()

    try:
        json_start = output_text.find("{")
        json_end = output_text.rfind("}") + 1
        parsed = json.loads(output_text[json_start:json_end])
    except Exception as e:
        print("❌ JSON 파싱 실패:", e)
        print("📦 원문 출력:", repr(output_text))
        parsed = {'fail'}

    return parsed, round(end - start, 2)


In [ ]:
#qwen 인퍼런스
messages = unified_system_prompt(text)
prompt = qwen_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
qwen_return, time_taken  = run_inference(prompt,qwen_tokenizer, qwen)
print("\n qwen:",phi_return,'시간:', time_taken )

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



LLM 원문 출력:
system

당신은 사용자의 금융 발화를 분석하는 AI 송금 도우미입니다. 다음 지침에 따라 작동하세요:

1. 사용자의 문장에서 다음 항목을 추출하세요:
    - intent: 다음 중 하나 (transfer, confirm, cancel, inquiry, other, system_response)
    - amount: 숫자만 추출 (없으면 null)
    - recipient: 사람 이름 등 (없으면 null)

2. 사용자의 발화에 어울리는 자연스러운 안내 응답(response)을 생성하세요.

3. 다음 JSON 형식으로 하나의 객체로 응답하세요. 다른 텍스트는 출력하지 마세요.

예시:
{
  "intent": "transfer",
  "amount": 30000,
  "recipient": "엄마",
  "response": "엄마님께 30,000원을 송금해드릴까요?"
}

user
아빠한테 오천 원 보내줘
assistant
아빠한테 오천 원 보내줘adge
蹾
蹾
蹾
蹾
蹾
蹾
蹾
蹾
蹾
蹾
蹾
蹾
蹾
蹾
蹾
蹾
蹾
蹾
蹾
蹾
蹾
蹾
蹾
蹾
蹾
蹾
蹾
蹾
蹾
蹾
蹾
蹾
蹾
蹾
蹾

처리 시간: 1.61초

 qwen: ({'fail'}, 2.3) 시간: 1.61


# Gemma


In [ ]:
# optimum-cli onnx \
#  --model "bartowski/gemma-2-2b-it" \
#  --task text-generation \
#  --optimize \
#  --static \
#  --output ./path_to_save

SyntaxError: invalid decimal literal (545287521.py, line 2)

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, convert_and_export_with_cache
#!pip cache purge 캐쉬 삭제


# 모델 설정
hf_model_repo = "google/gemma-2-2b-it"  # 예시 모델, 원하는 모델로 변경
cache_implementation = "static"  # StaticCache 사용
max_cache_len = 512  # 예시로 최대 캐시 길이를 512로 설정
batch_size = 1  # 예시로 배치 크기 설정

# 모델 불러오기
model = AutoModelForCausalLM.from_pretrained(
    hf_model_repo,
    attn_implementation="sdpa",  # Sparse Deterministic Partial Attention 사용
    generation_config=GenerationConfig(
        use_cache=True,
        cache_implementation=cache_implementation,
        max_length=max_cache_len,
        cache_config={
            "batch_size": batch_size,
            "max_cache_len": max_cache_len,
        },
    ),
)

RuntimeError: Failed to import transformers.integrations.executorch because of the following error (look up to see its traceback):
Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
Detected that PyTorch and torchvision were compiled with different CUDA major versions. PyTorch has CUDA Version=12.1 and torchvision has CUDA Version=11.8. Please reinstall the torchvision that matches your PyTorch install.

In [ ]:
# `convert_and_export_with_cache`를 사용하여 모델을 최적화하고 변환
exported_prog = convert_and_export_with_cache(model)

# ExecuTorch로 프로그램을 최적화하여 내보내기
# (저장된 프로그램을 ExecuTorch 형식으로 변환하여 성능 최적화)
# 예시로 `ExecuTorch`로 내보내는 코드 (실제 구현에 맞춰 변경 가능)
print("Exported program:", exported_prog)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_id="google/gemma-2-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id) 
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float32)
# # CUDA 사용 가능 여부 확인
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to("cuda")
model.eval() 

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

## onnx 변환

In [21]:
import torch  
# 예시 입력 문장 
sample_input = "Translate this to English: 안녕하세요"  
inputs = tokenizer(sample_input, return_tensors="pt").to("cuda")
dummy_input = inputs["input_ids"]

In [30]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, convert_and_export_with_cache

# 모델 설정
hf_model_repo = "google/gemma-2b-it"  # 예시 모델, 원하는 모델로 변경
cache_implementation = "static"  # StaticCache 사용
max_cache_len = 512  # 예시로 최대 캐시 길이를 512로 설정
batch_size = 1  # 예시로 배치 크기 설정

# 모델 불러오기
model = AutoModelForCausalLM.from_pretrained(
    hf_model_repo,
    attn_implementation="sdpa",  # Sparse Deterministic Partial Attention 사용
    generation_config=GenerationConfig(
        use_cache=True,
        cache_implementation=cache_implementation,
        max_length=max_cache_len,
        cache_config={
            "batch_size": batch_size,
            "max_cache_len": max_cache_len,
        },
    ),
)

# `convert_and_export_with_cache`를 사용하여 모델을 최적화하고 변환
exported_prog = convert_and_export_with_cache(model)

# ExecuTorch로 프로그램을 최적화하여 내보내기
# (저장된 프로그램을 ExecuTorch 형식으로 변환하여 성능 최적화)
# 예시로 `ExecuTorch`로 내보내는 코드 (실제 구현에 맞춰 변경 가능)
print("Exported program:", exported_prog)

# 예시로 실행할 입력
tokenizer = AutoTokenizer.from_pretrained(hf_model_repo)
input_text = "Translate this sentence to French."
inputs = tokenizer(input_text, return_tensors="pt")

# 모델 실행 (ExecuTorch 최적화된 버전으로)
# 실제로 ExecuTorch 라이브러리와 함께 실행되는 방식으로 추론 실행
# 더 빠른 성능을 위해 ExecuTorch에 최적화된 추론 구현 필요
outputs = exported_prog(inputs["input_ids"])  # 추론 실행

print(outputs)


ImportError: PyTorch SDPA requirements in Transformers are not met. Please install torch>=2.1.1.

In [ ]:
#gemma 인퍼런스
gemma_return = run_inference(prompt,gemma_processor, gemma)
print("\ngemma",gemma_return)

In [ ]:
#정적 계산 변환 JIT방식
torch.onnx.export(
    traced_model,
    (dummy_input,),
    "gemma-2b-it.onnx",
    input_names=["input_ids"],
    output_names=["logits"],
    opset_version=13
    # do_constant_folding=True,
    # dynamic_axes={"input_ids": {0: "batch_size", 1: "sequence_length"}, "logits": {0: "batch_size", 1: "sequence_length"}} 
)

RuntimeError: Tracer cannot infer type of CausalLMOutputWithPast(loss=None, logits=tensor([[[-23.4456,  15.3849, -25.9747,  ..., -22.8236, -17.0587, -22.6181],
         [-23.4456,  15.3849, -25.9747,  ..., -22.8236, -17.0587, -22.6181],
         [-23.4456,  15.3849, -25.9747,  ..., -22.8236, -17.0587, -22.6181],
         ...,
         [-23.4456,  15.3849, -25.9747,  ..., -22.8236, -17.0587, -22.6181],
         [-23.4456,  15.3849, -25.9747,  ..., -22.8236, -17.0587, -22.6181],
         [-23.4456,  15.3849, -25.9747,  ..., -22.8236, -17.0587, -22.6181]]],
       device='cuda:0', grad_fn=<UnsafeViewBackward0>), past_key_values=<transformers.cache_utils.DynamicCache object at 0x743026f2e800>, hidden_states=None, attentions=None)
:Only tensors and (possibly nested) tuples of tensors, lists, or dictsare supported as inputs or outputs of traced functions, but instead got value of type DynamicCache.

In [ ]:
#gemma 인퍼런스
gemma_return = run_inference(prompt,gemma_processor, gemma)
print("\ngemma",gemma_return)